In [2]:
print("hello")

hello


In [26]:
%pwd

'e:\\Downloads\\New folder\\Medical-Chatbot\\research'

In [27]:
import os
os.chdir("../")

In [28]:
%pwd

'e:\\Downloads\\New folder\\Medical-Chatbot'

In [29]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [3]:
import os
print(os.getcwd())

e:\Downloads\New folder\Medical-Chatbot\research


In [9]:
!pip install pypdf

Defaulting to user installation because normal site-packages is not writeable
  Using cached pypdf-5.7.0-py3-none-any.whl.metadata (7.2 kB)
Using cached pypdf-5.7.0-py3-none-any.whl (305 kB)


In [10]:
extracted_data = load_pdf_file(r"E:\Downloads\New folder\Medical-Chatbot\Data")

In [14]:
#extracted_data

Chunking operation

In [14]:
#split the text into text chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

# Now run the function
text_chunks = text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 40000


In [17]:
from langchain.embeddings import HuggingFaceEmbeddings

In [18]:
#download the embadding hugging face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MinLM-L6-v2')
    return embeddings

In [19]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\HP Laptop\AppData\Local\Temp\ipykernel_24044\3415835077.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
C:\Users\HP Laptop\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
query_result = embeddings.embed_query("Hello World")
print("Length",len(query_result))

Length 384


In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [24]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("PINECONE_API_KEY")

# Optional: remove quotes if they're still there
api_key = api_key.strip('"')

print("API KEY:", api_key)

API KEY: pcsk_Km16b_311aZo4c6ZT9JtHv4ruihKWtSmQgpCEgbURyCVm27Fhxjqywo2R54yQnptLyiHD


In [25]:
from pinecone import Pinecone

pc = Pinecone(api_key=api_key)

# Connect to existing index
index = pc.Index("medicalbot")

# You can now describe or use the index
stats = index.describe_index_stats()
print("Total vectors stored:", stats.total_vector_count)

Total vectors stored: 0


In [27]:
from dotenv import load_dotenv
import os
from pinecone import Pinecone

load_dotenv()  # ✅ Reads from .env file

api_key = os.getenv("PINECONE_API_KEY")
print("Using API Key:", api_key)  # Should show the actual long key

pc = Pinecone(api_key=api_key)

index = pc.Index("medicalbot")
stats = index.describe_index_stats()
print("Total vectors stored:", stats.total_vector_count)

Using API Key: pcsk_Km16b_311aZo4c6ZT9JtHv4ruihKWtSmQgpCEgbURyCVm27Fhxjqywo2R54yQnptLyiHD
Total vectors stored: 0


In [64]:
import os
os.environ["PINECONE_API_KEY"] = "pcsk_Km16b_311aZo4c6ZT9JtHv4ruihKWtSmQgpCEgbURyCVm27Fhxjqywo2R54yQnptLyiHD"
import os
os.environ["COHERE_API_KEY"] = "NBP7WXJCUucInhPHSJMfiPSlz2TKC3mTCr1VxxmP"

In [33]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.schema import Document

# Optional: if your text_chunks are not in Document format
if isinstance(text_chunks[0], str):
    text_chunks = [Document(page_content=chunk) for chunk in text_chunks]

# ✅ Define the embedding model
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# ✅ Upload documents to Pinecone
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embedding,
    index_name="medicalbot"
)

In [34]:
# Import the correct class
from langchain_pinecone import PineconeVectorStore

# Use the existing index and embedding model
docsearch = PineconeVectorStore.from_existing_index(
    index_name="medicalbot",         # ✅ Make sure this matches your index
    embedding=embedding              # ✅ This should be your embedding model object
)

In [35]:
docsearch

In [36]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [39]:
retrieved_docs = retriever.invoke("What is Acne?")

In [40]:
retrieved_docs

[Document(id='693bb00b-7453-4fec-ab23-e05e055e40d4', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'E:\\Downloads\\New folder\\Medical-Chatbot\\Data\\MedicalBook.pdf', 'total_pages': 4505.0}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(id='bda2a206-1148-480a-8aea-ead422e2637a', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'E:\\Downloads\\New folder\\Medical-Chatbot\\Data\\MedicalBook.pdf', 'total_pages': 4505.0}, page_content='Sebaceous follicles— A structure found within the\nskin that houses the oil-producing glands and hair\nfollicles, where pimples form.\nSebum— An oily skin moisturizer pro

In [65]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [68]:
!pip install langchain-cohere cohere

Defaulting to user installation because normal site-packages is not writeable
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)

  Attempting uninstall: httpx-sse

    Found existing installation: httpx-sse 0.4.1

    Uninstalling httpx-sse-0.4.1:

      Successfully uninstalled httpx-sse-0.4.1

   -------------------- ------------------- 3/6 [fastavro]
   -------------------- ------------------- 3/6 [fastavro]
   -------------------- ------------------- 3/6 [fastavro]
   -------------------------- ------------- 4/6 [cohere]
   -------------------------- ------------- 4/6 [cohere]
   -------------------------- ------------- 4/6 [cohere]
   -------------------------- ------------- 4/6 [cohere]
   -------------------------- ------------- 4/6 [cohere]
   -------------------------- ------------- 4/6 [cohere]
   -------------------------- ------------- 4/6 [cohere]
   -------------------------- ------------- 4/6 [cohere]

In [69]:
from langchain_cohere import ChatCohere

llm = ChatCohere(model="command-r", temperature=0.4)

response = llm.invoke("What is the capital of France?")
print(response.content)

The capital of France is Paris.


In [70]:
import os
from langchain_cohere import ChatCohere

# Set your Cohere API key
os.environ["COHERE_API_KEY"] = "NBP7WXJCUucInhPHSJMfiPSlz2TKC3mTCr1VxxmP"

# Initialize the Cohere chat model
llm = ChatCohere(model="command-r", temperature=0.4)

# Run a prompt
response = llm.invoke("What is the capital of France?")
print(response.content)

The capital of France is Paris.


In [72]:
from langchain_cohere import ChatCohere
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import os

# ✅ Set Cohere API key
os.environ["COHERE_API_KEY"] = "NBP7WXJCUucInhPHSJMfiPSlz2TKC3mTCr1VxxmP"

# ✅ Load your embedding model and Pinecone vector store (already uploaded earlier)
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
docsearch = PineconeVectorStore.from_existing_index(
    index_name="medicalbot",
    embedding=embedding
)

# ✅ Define the retriever
retriever = docsearch.as_retriever()

# ✅ Initialize Cohere LLM
llm = ChatCohere(model="command-r", temperature=0.4)

# ✅ Prompt for retrieval-based QA
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are an assistant for question-answering tasks. "
     "Use the following pieces of retrieved context to answer the question. "
     "If you don't know the answer, say you don't know. "
     "Answer concisely in three sentences.\n\n{context}"),
    ("human", "{input}")
])

# ✅ Create the document QA chain
document_chain = create_stuff_documents_chain(llm, prompt)

# ✅ Combine with retriever
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# ✅ Example usage
response = retrieval_chain.invoke({"input": "What is diabetes?"})
print("Answer:", response["answer"])

Answer: Diabetes mellitus is a chronic condition where the body cannot absorb glucose into its cells due to insufficient or ineffective insulin. This energy source shortage manifests as a host of symptoms, including frequent urination, extreme hunger and thirst, and lethargy. Treatment involves dietary changes, medication, and sometimes insulin injections.


In [73]:
response = retrieval_chain.invoke({"input": "What is Acne?"})
print("Answer:", response["answer"])

Answer: Acne is a skin disease that causes pimples and inflammation on the face, chest, and back. It happens when hair follicles become blocked by oil, dead skin cells, and bacteria. This blockage is called sebum and it collects inside the pores, leading to acne lesions.
